# Drive Test Tag Generation With BERTopic
Generate tags for the written portion of the chinese driving exam using BERTopic.

## 1. Load Data
Loading data fom local database into a pandas dataframe

### a) Load data into question bank class

In [1]:
from qb.question import Question
from qb.question_bank import QuestionBank
from data_storage.database.json_database import LocalJsonDB

db = LocalJsonDB("data_storage/database/json_db/data.json",
                 "data_storage/database/json_db/images")
qb : QuestionBank = db.load()
print(qb.question_count())

2836


### b) Convert question bank to pandas dataframe

In [2]:
import pandas as pd
from pandas import DataFrame
def qb_to_df(qb: QuestionBank) -> DataFrame:
    data = {
        "ID": [],
        "Question": [],
        "Answer Choices": [],
        "Answer": [],
        "Chapter": [],
        "Image Path": []
    }
    for chapter_id in qb.get_all_chapter_num():
        chapter_description = qb.describe_chapter(chapter_id)
        qid_lst = qb.get_qids_by_chapter(chapter_id)
        for qid in qid_lst:
            question: Question = qb.get_question(qid)
            data["ID"].append(qid)
            data["Question"].append(question.get_question())
            data["Answer Choices"].append(", ".join(question.get_answers()))
            data["Answer"].append(question.get_correct_answer())
            data["Chapter"].append(chapter_description)
            data["Image Path"].append(question.get_img_path() if question.get_img_path() else "")
    return pd.DataFrame(data)
question_bank = qb_to_df(qb)

In [4]:
print(question_bank.shape)
question_bank.head()

(2836, 6)


,ID,Question,Answer Choices,Answer,Chapter,Image Path
0,7d9ae,驾驶机动车在高速公路上行驶，遇低能见度气象条件时，能见度在200米以下，车速不得超过每小时多...,"40，80, 70，100, 30，80, 60，100",60，100,道路交通安全法律、法规和规章,
1,be189,驾驶机动车载货长度、宽度、高度超过规定的，一次记1分。,"对, 错",对,道路交通安全法律、法规和规章,
2,ed1cc,驾驶机动车应在变更车道的同时开启转向灯。,"对, 错",错,道路交通安全法律、法规和规章,
3,fd850,车辆上道路行驶前，驾乘人员都应当系好安全带。,"对, 错",对,道路交通安全法律、法规和规章,
4,ed1b3,小型汽车驾驶人有下列哪种交通违法行为，一次记3分？,"在高速公路、城市快速路上行驶超速50%以上的, 在高速公路、城市快速路上行驶超速20%以上未...",在普通道路上行驶超速20%以上未达到50%的,道路交通安全法律、法规和规章,


## 2.